<a href="https://colab.research.google.com/github/Rakesh2497/jovian/blob/main/Chapter1_Practice_Session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/customers.csv
!wget https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/products.csv
!wget https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/orders.csv
!wget https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/order_items.csv
!wget https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/employees.csv
!wget https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/transactions.csv

--2025-06-13 21:41:23--  https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1780 (1.7K) [text/plain]
Saving to: ‘customers.csv’

customers.csv       100%[===================>]   1.74K  --.-KB/s    in 0s      

2025-06-13 21:41:23 (25.6 MB/s) - ‘customers.csv’ saved [1780/1780]

--2025-06-13 21:41:24--  https://raw.githubusercontent.com/icyanide9/de-practice/refs/heads/main/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 806 [text/plain]

In [2]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("PySpark Data Engineering Practice").getOrCreate()

# Load the files into dataframe
customers_df = spark.read.csv("customers.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)
orders_df = spark.read.csv("orders.csv", header=True, inferSchema=True)
order_items_df = spark.read.csv("order_items.csv", header=True, inferSchema=True)
employees_df = spark.read.csv("employees.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)

In [3]:
from os import truncate
print(f"The count before applying DropDuplicate Funciton: {customers_df.count()}")



cust_drop_dup = customers_df.dropDuplicates()

print(f"The count before applying DropDuplicate Funciton: {cust_drop_dup.count()}")


#dropping null records,

cust_drop_nulls = customers_df.dropna()
cust_drop_nulls.show(truncate=False)
cust_drop_nulls.count()


cust_drop_nulls_subset = customers_df.dropna(subset=["name", "phone"])
print(f"Subset Drop nulls Count: {cust_drop_nulls_subset.count()}")


#na.replace

cust_na_replace = customers_df.na.replace(
    {"john.doe@example.com": "john.doe@gmail.com", "jane.smith@example.com": "jane.smith@gmail.com"},
    subset=["email"]
)

cust_na_replace.show()
cust_na_replace.count()



The count before applying DropDuplicate Funciton: 22
The count before applying DropDuplicate Funciton: 22
+-----------+---------------+---------------------------+--------+---------------+-----------------+--------+
|customer_id|name           |email                      |phone   |address        |registration_date|status  |
+-----------+---------------+---------------------------+--------+---------------+-----------------+--------+
|1          |John Doe       |john.doe@example.com       |555-1234|123 Elm St     |2022-01-15       |Active  |
|2          |Jane Smith     |jane.smith@example.com     |555-5678|456 Oak St     |2021-11-20       |Active  |
|3          |Michael Brown  |michael.brown@example.com  |555-8765|789 Pine St    |2023-02-10       |Inactive|
|4          |Emily Davis    |emily.davis@example.com    |555-4321|321 Maple St   |2022-05-05       |Active  |
|5          |David Wilson   |david.wilson@example.com   |555-2468|654 Cedar St   |2021-09-12       |Active  |
|7          |C

22

In [4]:
#Type Conversions
from pyspark.sql.functions import *
import pyspark.sql.functions as F

customers_df = customers_df.withColumn("registration_date", to_date(col("registration_date"), "yyyy/MM/dd"))
customers_df.show()

+-----------+---------------+--------------------+--------+---------------+-----------------+--------+
|customer_id|           name|               email|   phone|        address|registration_date|  status|
+-----------+---------------+--------------------+--------+---------------+-----------------+--------+
|          1|       John Doe|john.doe@example.com|555-1234|     123 Elm St|       2022-01-15|  Active|
|          2|     Jane Smith|jane.smith@exampl...|555-5678|     456 Oak St|       2021-11-20|  Active|
|          3|  Michael Brown|michael.brown@exa...|555-8765|    789 Pine St|       2023-02-10|Inactive|
|          4|    Emily Davis|emily.davis@examp...|555-4321|   321 Maple St|       2022-05-05|  Active|
|          5|   David Wilson|david.wilson@exam...|555-2468|   654 Cedar St|       2021-09-12|  Active|
|          6|  Sarah Johnson|                NULL|555-1357|   987 Birch St|       2022-07-18|Inactive|
|          7|      Chris Lee|chris.lee@example...|555-9753|  159 Spruce S

In [5]:
#type_covertion: price
products_df.show()

prod_df_type_change = products_df.withColumn("price", col("price").cast("float"))

prod_df_type_change.printSchema()

#orders dataframe
orders_df.show()
orders_df.printSchema()

order_df_type_change = orders_df.withColumn("order_amount", col("order_amount").cast("float"))

order_df_type_change.printSchema()


#Transactions dataframe
# transactions_df.show()

transac_df_type_change = transactions_df.withColumn("created_at", to_timestamp(col("created_at"), "yyyy-MM-dd HH:mm:ss"))

transac_df_type_change.printSchema()



+----------+--------------+-----------+------+--------------+
|product_id|  product_name|   category| price|stock_quantity|
+----------+--------------+-----------+------+--------------+
|         1|Wireless Mouse|Electronics| 25.99|           150|
|         2|  USB Keyboard|Electronics| 19.99|           120|
|         3|    HD Monitor|Electronics|149.99|            80|
|         4|     Desk Lamp|  Furniture| 34.99|            60|
|         5|  Office Chair|  Furniture| 89.99|            40|
|         6|      Notebook| Stationery|  2.99|           500|
|         7| Ballpoint Pen| Stationery|  0.99|          1000|
|         8|  Water Bottle|Accessories| 12.99|           200|
|         9|      Backpack|Accessories| 39.99|            90|
|        10|    Smartphone|Electronics|399.99|            30|
|        11|        Tablet|Electronics|299.99|            25|
|        12|Desk Organizer|  Furniture| 15.99|            70|
|        13|  Sticky Notes| Stationery|  1.99|           600|
|       

In [6]:
#String Manipulation
from pyspark.sql.functions import col, upper, substr, trim, split, lit # Import trim and split here

#creating new column with has the domain name of the email along with trimming the name column.

changes_to_make = {"name": trim(col("name")), "domain": split(col("email"), "@")[1]}

# Apply changes to customers_df by creating a new dataframe
updated_customer_df = customers_df
for col_name, expr in changes_to_make.items():
  updated_customer_df = updated_customer_df.withColumn(col_name, expr)

updated_customer_df.show()




products_df.show()

products_dfs = products_df.withColumn("Short_Name", substr(upper(col("product_name")), lit(1), lit(5)))

products_dfs.show()

+-----------+---------------+--------------------+--------+---------------+-----------------+--------+-----------+
|customer_id|           name|               email|   phone|        address|registration_date|  status|     domain|
+-----------+---------------+--------------------+--------+---------------+-----------------+--------+-----------+
|          1|       John Doe|john.doe@example.com|555-1234|     123 Elm St|       2022-01-15|  Active|example.com|
|          2|     Jane Smith|jane.smith@exampl...|555-5678|     456 Oak St|       2021-11-20|  Active|example.com|
|          3|  Michael Brown|michael.brown@exa...|555-8765|    789 Pine St|       2023-02-10|Inactive|example.com|
|          4|    Emily Davis|emily.davis@examp...|555-4321|   321 Maple St|       2022-05-05|  Active|example.com|
|          5|   David Wilson|david.wilson@exam...|555-2468|   654 Cedar St|       2021-09-12|  Active|example.com|
|          6|  Sarah Johnson|                NULL|555-1357|   987 Birch St|     

In [20]:
 # Filtering and Selection

from pyspark.sql.functions import *

orders_df.show()

orders_df.filter((col("order_amount")>"100") & (col("order_status") == "Completed")).orderBy(col("order_id")).show()

#second_filer

customers_df.filter((col("registration_date") > lit("2022-01-01")) & (col("status") == "Active")).show()



+--------+-----------+----------+------------+------------+--------------+
|order_id|customer_id|order_date|order_amount|order_status|payment_method|
+--------+-----------+----------+------------+------------+--------------+
|    1001|          1|2023-01-10|      120.97|   Completed|   Credit Card|
|    1002|          2|2023-01-12|       59.98|   Completed|        PayPal|
|    1003|          3|2023-01-15|      399.99|     Pending|   Credit Card|
|    1004|          4|2023-01-18|       89.99|   Completed|    Debit Card|
|    1005|          5|2023-01-20|       15.97|   Cancelled|   Credit Card|
|    1006|          6|2023-01-22|      299.99|   Completed|        PayPal|
|    1007|          7|2023-01-25|       24.99|   Completed|   Credit Card|
|    1008|          8|2023-01-28|      149.99|     Pending|    Debit Card|
|    1009|          9|2023-02-01|       34.99|   Completed|   Credit Card|
|    1010|         10|2023-02-03|       59.98|   Completed|        PayPal|
|    1011|         11|202

In [25]:
#5. Creating New Columns

from pyspark.sql.functions import *
customers_df.show()

customers_df = customers_df.withColumn("tenure_year", 2025 - year(col("registration_date")))

customers_df.show()


orders_df.show()

orders_df = orders_df.withColumn("is_high_value", when(col("order_amount") > 250, "True").otherwise("False"))

orders_df.show()

+-----------+---------------+--------------------+--------+---------------+-----------------+--------+-----------+
|customer_id|           name|               email|   phone|        address|registration_date|  status|tenure_year|
+-----------+---------------+--------------------+--------+---------------+-----------------+--------+-----------+
|          1|       John Doe|john.doe@example.com|555-1234|     123 Elm St|       2022-01-15|  Active|          3|
|          2|     Jane Smith|jane.smith@exampl...|555-5678|     456 Oak St|       2021-11-20|  Active|          4|
|          3|  Michael Brown|michael.brown@exa...|555-8765|    789 Pine St|       2023-02-10|Inactive|          2|
|          4|    Emily Davis|emily.davis@examp...|555-4321|   321 Maple St|       2022-05-05|  Active|          3|
|          5|   David Wilson|david.wilson@exam...|555-2468|   654 Cedar St|       2021-09-12|  Active|          4|
|          6|  Sarah Johnson|                NULL|555-1357|   987 Birch St|     

In [28]:
#6. Renaming Columns

customers_df.show(5)

customers_df = customers_df.withColumnRenamed("name", "customer_name")

customers_df.show(5)


orders_df.show(5)

orders_df = orders_df.withColumnRenamed("order_amount", "total_amount")

orders_df.show(5)


+-----------+-------------+--------------------+--------+------------+-----------------+--------+-----------+
|customer_id|customer_name|               email|   phone|     address|registration_date|  status|tenure_year|
+-----------+-------------+--------------------+--------+------------+-----------------+--------+-----------+
|          1|     John Doe|john.doe@example.com|555-1234|  123 Elm St|       2022-01-15|  Active|          3|
|          2|   Jane Smith|jane.smith@exampl...|555-5678|  456 Oak St|       2021-11-20|  Active|          4|
|          3|Michael Brown|michael.brown@exa...|555-8765| 789 Pine St|       2023-02-10|Inactive|          2|
|          4|  Emily Davis|emily.davis@examp...|555-4321|321 Maple St|       2022-05-05|  Active|          3|
|          5| David Wilson|david.wilson@exam...|555-2468|654 Cedar St|       2021-09-12|  Active|          4|
+-----------+-------------+--------------------+--------+------------+-----------------+--------+-----------+
only showi

In [36]:
# 7. Sorting Data
from pyspark.sql.functions import *

products_df.show(5)

products_df = products_df.orderBy(col("price"), ascending=False)

products_df.show(5)

##########
customers_df.show(5)

customers_df = customers_df.orderBy(col("tenure_year"), ascending=False)
customers_df.show(10)




+----------+------------+-----------+------+--------------+
|product_id|product_name|   category| price|stock_quantity|
+----------+------------+-----------+------+--------------+
|        10|  Smartphone|Electronics|399.99|            30|
|        11|      Tablet|Electronics|299.99|            25|
|         3|  HD Monitor|Electronics|149.99|            80|
|        14|File Cabinet|  Furniture|129.99|            20|
|         5|Office Chair|  Furniture| 89.99|            40|
+----------+------------+-----------+------+--------------+
only showing top 5 rows

+----------+------------+-----------+------+--------------+
|product_id|product_name|   category| price|stock_quantity|
+----------+------------+-----------+------+--------------+
|        10|  Smartphone|Electronics|399.99|            30|
|        11|      Tablet|Electronics|299.99|            25|
|         3|  HD Monitor|Electronics|149.99|            80|
|        14|File Cabinet|  Furniture|129.99|            20|
|         5|Off

In [57]:
#8. Sampling Data

sample_cust_df = customers_df.sample(fraction=0.2, seed=100)
sample_cust_df.show()
print(customers_df.count())

sample_order_df = orders_df.sample(fraction=0.2, seed=100)
sample_order_df.show()
orders_df.count()

+-----------+--------------+--------------------+--------+---------------+-----------------+--------+-----------+
|customer_id| customer_name|               email|   phone|        address|registration_date|  status|tenure_year|
+-----------+--------------+--------------------+--------+---------------+-----------------+--------+-----------+
|          5|  David Wilson|david.wilson@exam...|555-2468|   654 Cedar St|       2021-09-12|  Active|          4|
|         20|   Chloe Perez|chloe.perez@examp...|555-6841| 369 Dogwood St|       2021-07-22|Inactive|          4|
|          1|      John Doe|john.doe@example.com|555-1234|     123 Elm St|       2022-01-15|  Active|          3|
|          4|   Emily Davis|emily.davis@examp...|555-4321|   321 Maple St|       2022-05-05|  Active|          3|
|         15|Benjamin Adams|benjamin.adams@ex...|555-7412|369 Magnolia St|       2022-12-20|  Active|          3|
+-----------+--------------+--------------------+--------+---------------+--------------

22

In [60]:
#9. Date Handling and Transformations

from pyspark.sql.functions import *

operations = {"month": month(col("created_at")), "day": day(col("created_at")), "year": year(col("created_at"))}

for column_name, operation in operations.items():
  transactions_df = transactions_df.withColumn(column_name, operation)

transactions_df.show(5)

transactions_df = transactions_df.withColumn("days_diff", datediff(col("transaction_date"), col("created_at")))

transactions_df.show()

+--------------+-----------+----------------+------+----------------+-----------+-------------------+-----+---+----+
|transaction_id|customer_id|transaction_date|amount|transaction_type|   location|         created_at|month|day|year|
+--------------+-----------+----------------+------+----------------+-----------+-------------------+-----+---+----+
|          5001|          1|      2023-01-10|120.97|        Purchase|   New York|2023-01-10 09:15:23|    1| 10|2023|
|          5002|          2|      2023-01-12| 59.98|        Purchase|Los Angeles|2023-01-12 10:05:10|    1| 12|2023|
|          5003|          3|      2023-01-15|399.99|        Purchase|    Chicago|2023-01-15 14:22:45|    1| 15|2023|
|          5004|          4|      2023-01-18| 89.99|        Purchase|    Houston|2023-01-18 16:30:00|    1| 18|2023|
|          5005|          5|      2023-01-20| 15.97|          Refund|    Phoenix|2023-01-20 11:00:00|    1| 20|2023|
+--------------+-----------+----------------+------+------------

In [64]:
#10. Date Range Filtering

from pyspark.sql.functions import *

orders_btw_dates = orders_df.filter((col("order_date")>="2023-01-20") & (col("order_date")<="2023-02-05"))

orders_btw_dates.show()


trans_in_feb = transactions_df.filter((col("transaction_date")>= lit("2023-02-01")) & (col("transaction_date")<= lit("2023-02-28")))

trans_in_feb.show()

+--------+-----------+----------+------------+------------+--------------+-------------+
|order_id|customer_id|order_date|total_amount|order_status|payment_method|is_high_value|
+--------+-----------+----------+------------+------------+--------------+-------------+
|    1005|          5|2023-01-20|       15.97|   Cancelled|   Credit Card|        False|
|    1006|          6|2023-01-22|      299.99|   Completed|        PayPal|         True|
|    1007|          7|2023-01-25|       24.99|   Completed|   Credit Card|        False|
|    1008|          8|2023-01-28|      149.99|     Pending|    Debit Card|        False|
|    1009|          9|2023-02-01|       34.99|   Completed|   Credit Card|        False|
|    1010|         10|2023-02-03|       59.98|   Completed|        PayPal|        False|
|    1011|         11|2023-02-05|       89.99|   Completed|   Credit Card|        False|
+--------+-----------+----------+------------+------------+--------------+-------------+

+--------------+----